In [7]:
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# import sys
# os.chdir('/content/drive/My Drive/ProyectoIA2/generative-spectral-datasets')

In [18]:
import tensorflow as tf
import numpy as np
from models.main import make_autoencoder, make_discriminator


input_shape = (28,28,1)

In [26]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = train_images / 255

BUFFER_SIZE = 60000
BATCH_SIZE = 256

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [20]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
mse_loss = tf.keras.losses.MeanSquaredError()


def autoencoder_loss(real_x, estimated_x, real_output, fake_output):
    autoencoder_loss = mse_loss(real_x, estimated_x)
    gan_loss =  discriminator_loss(real_output, fake_output)
    total_loss = autoencoder_loss - gan_loss
    return total_loss


def discriminator_loss(real_output, fake_output):
    loss = tf.math.log(real_output) + tf.math.log(1 - fake_output)
    return - tf.math.reduce_mean(loss)

autoencoder_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


autoencoder = make_autoencoder(input_shape)
discriminator = make_discriminator(input_shape)

In [22]:
@tf.function
def train_step(images):

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = autoencoder(images, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = autoencoder_loss(images, generated_images, real_output, fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, autoencoder.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    autoencoder_optimizer.apply_gradients(zip(gradients_of_generator, autoencoder.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


def train(dataset, epochs):
  print("START TRAINING")
  for epoch in range(epochs):
    for image_batch in dataset:
      train_step(image_batch)
  print("END TRAINING")


In [31]:
EPOCHS = 10


train(train_dataset, EPOCHS)